##### Import Package

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import konlpy.tag 
import pandas as pd
import scipy as sp
from scipy import spatial
import operator
import numpy as np

##### Get Article & Comment

In [2]:
def shuffle_df(df):
    return df.iloc[np.random.permutation(len(df))].reset_index(drop=True) 

def get_datas(date, comment=1):
    article_df = pd.read_csv("../data/news/" + date + ".csv")
    article_df = shuffle_df(article_df)
    comment_df = pd.read_csv("../data/comment/comment"+str(comment)+".csv")
    return article_df, comment_df

#####  Morpheme Analytics

In [3]:
def make_document_morpheme(texts):
    """make documents list after morphological"""
    
    documents = []
    
    komoran = konlpy.tag.Komoran()
    
    for text in texts:
        obj = line2obj(komoran.pos(text, flatten=False))
        documents.append(obj)
        
    return documents

def line2obj(lines):
    """ make keywords dictionary include only (NNP, NNG)"""
    
    obj = {}
    
    for line in lines:
        for keyword in line:
            if len(keyword[0]) > 1 and ( keyword[1] == "NNP" or keyword[1] == "NNG" ) :
                key = keyword[0]
                if key in obj:
                    obj[key] += 1
                else:
                    obj[key] = 1
                    
    return obj

In [4]:
def gen_text_documents(documents):
    
    text_documents = []
    
    for document in documents:
        
        text_words = []
        
        for word, count in document.items():
            text_words.extend([word] * count)
            
        text_document = " ".join(text_words)
        text_documents.append(text_document)
        
    return text_documents        

In [5]:
def calc_euc_dist(v1, v2):
    delta = v1 - v2
    dist = sp.linalg.norm(delta.toarray())
    return dist

def calc_cos_dist(v1, v2):
#     v1 = v1.toarray()
#     v2 = v2.toarray()
#     dist = 1.0 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    dist = spatial.distance.cosine(v1.toarray(), v2.toarray())
    
    return dist

def get_euc_dists(vectorized, num_docs):
    
    dists = []
    
    for idx in range(num_docs-1):
#         dist = calc_euc_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
        dist = calc_cos_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
        dists.append((num_docs-1, idx, dist))
        
    return dists

def sort_dists(dists):
    result_list = []
    for i, j, dist in sorted(dists, key=operator.itemgetter(2)):
        result_list.append((i, j, dist))
    return result_list

##### Text Preprocessing

In [6]:
# get data
article_df, comment_df = get_datas("2016-07-06", 1)
len(article_df), len(comment_df)

(2241, 15)

In [14]:
# make documents
comment_str = comment_df["content"][:].str.cat(sep=' ')
article_series = article_df["content"][:2000]
article_series = article_series.append(pd.Series([comment_str])).reset_index(drop=True)
%time article_documents = make_document_morpheme(article_series)

CPU times: user 36.2 s, sys: 449 ms, total: 36.7 s
Wall time: 31.1 s


In [15]:
# make gen documents
gen_articles = gen_text_documents(article_documents)

##### Vectorizer

In [16]:
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
# vectorizer = HashingVectorizer()

vectorized = vectorizer.fit_transform(gen_articles)
num_docs, num_features =  vectorized.shape
feature_names = vectorizer.get_feature_names()

print("the number of input documents : {}".format(num_docs))
print("the number of words : {}".format(num_features))
result_df = pd.DataFrame(vectorized.toarray(), columns=feature_names)

the number of input documents : 2001
the number of words : 22958


##### Measurement Distance between comment and document

In [17]:
dists = get_euc_dists(vectorized, num_docs)
result = sort_dists(dists)
result[:10]

[(2000, 1912, 0.86779432516931476),
 (2000, 1658, 0.86837712806770639),
 (2000, 1550, 0.87407558097722782),
 (2000, 217, 0.8861027940442362),
 (2000, 671, 0.88651258206522221),
 (2000, 1163, 0.88861011320450345),
 (2000, 1039, 0.90234043545470832),
 (2000, 1127, 0.91141615323066449),
 (2000, 998, 0.91672520814115388),
 (2000, 636, 0.91992304322809038)]

In [26]:
# comment
print(comment_str)

영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 우선 오늘 저녁8부터는 다시 항공편 운항 재개를 한다고 하니 조금만 더 기다리시면 될 것 같아요. 제주도 관계자분들도 힘내세요! 영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 아울러 대구 지하철 참사를 다시 생각하게 해주셔서 감사합니다. 제주도를 비롯해서 다른 지역 충청이남서해안 지역분들과 전라도 부근 주민들도 힘내세요! 눈 때문에 고생이 많으세요! 그래도 자수를 했다니 다행이네요, 피해자분들도 마음 잘 추수리셔서 화이팅하셨으면 좋겠습니다. 경찰분들도 바쁘신건 알지만 힘 내주세요! 충북 제천지역도 다른지역과 같이 많이 추운가 봅니다ㅠ 농민분들이 엄청 고생하시는 것 같습니다. 조금만 있으면 날씨가 풀릴 것 같으니 힘내세요! 평창올림픽의 성공을 위하여 관련 관계자분들이 스페인 마드리드까지 가서 신다고 하는데 감사합니다. 평창올림픽의 성공적인 개최를 기원합니다. KTX 지연, 제주공항 운항 지연에 뒤이은 여러번의 동파사고까지, 조금만 지나면 날시가 풀릴테니 힘내세요!! 모쪼록 몸에 해로운 담배입니다. 다른 나라의 광고가 어떻든 대한민국 국민의 건강 증진을 위하여 금연을 기원하시는 분들은 금연에 성공하시길 바라겠습니다. 어린 아이들이 이렇게 모범을 보여주니 너무 고맙습니다. 화단정리에 노인들께 식사를 대접하기까지 너무 마음이 예쁩니다. 이렇게 노인장기요양서비스가 통합(통일)된다면 노인분들이 이용하시기에도 간편하고 좋은 서비스활동이 될 것 같습니다. 미래를 위해 방학중에 대학 공동 진로진학 박람회까지 가서 자신의 미래를 설꼐하는것을 보면 너무 아름다운 것 같습니다. 이 추운 날씨에도 고생해주시는 장병분들을 보면 감사한 마음 뿐입니다. 급식을 비롯해서 여러 여건들이 개선되길 희망합니다! 이렇게 추운날씨에 남들을 위해  쌀 1000kg을 기부해주신 대한불교조계종 문수사(주지 스님 혜안)께 너무 감사합니다! 이렇게 

##### Recomend Article

In [27]:
comment, rank_idx, distance = zip(*result)
idx = rank_idx[0]
print(idx)
print(article_series[idx].split("▶")[0])

1912
[글로벌 기업들 로봇 경쟁] 구글 - 알파고와 같은 인공지능으로 작동하는 정교한 로봇팔 훈련 중도요타 - 2020년 도우미 로봇 출시 계획현대차 - 웨어러블 로봇 시제품 공개LG전자 - 인천공항 안내 로봇 곧 선보여글로벌 기업들의 로봇 기술 경쟁이 뜨거워지고 있다. 지금까지는 생산 현장에서 단순 작업을 반복하는 공정 자동화용 로봇을 주로 활용해왔지만 최근에는 인공지능을 심은 서비스 로봇이 주목받고 있다. 시장이 본격적 성장 조짐을 보이면서 전자·소프트웨어는 물론 자동차 분야까지 세계적 기업이 잇따라 뛰어들기 시작했다. KAIST 오준호 교수(기계공학)는 "현재 로봇은 산업용 이외에 군사용 등 특수 목적으로만 일부 쓰이고 있지만 인공지능과 결합하면 다양한 서비스 로봇이 등장할 것"이라고 말했다.◇몸체 갖춘 '알파고'의 등장세계 최대 인터넷 기업 구글은 바둑 프로그램 '알파고'처럼 인공지능으로 작동하는 로봇을 개발 중이다. 알파고는 소프트웨어가 수를 계산해 내면 인간이 바둑돌을 대신 놓아줘야 한다. 구글은 인공지능에 로봇이라는 몸체를 결합해 직접 임무를 수행하도록 할 계획이다.구글이 개발 중인 인공지능 로봇 팔. 크기와 모양이 제각각인 여러 물건을 집게 손으로 정확하게 들어 올릴 수 있도록 했다. /구글구글은 우선 로봇 팔 개발에 나서고 있다. 알파고가 인간이 둔 기보(棋譜)를 학습해 바둑을 익힌 것처럼, 로봇 팔도 학습을 거쳐 더 정교하게 움직이도록 개발하는 것이다. 물건을 집었을 때 집게에 정확하게 잡혔는지, 아니면 헛손질을 했는지 등을 데이터화해 시행착오를 줄여나가고 있다.일본 소니는 최근 100억엔(약 1130억원) 규모 펀드를 조성해 로봇·인공지능 분야에 투자하기로 했다. 히라이 가즈오 소니 최고경영자(CEO)는 "애완견 로봇 아이보 사업을 재개하는 수준이 아니라, 로봇과 인공지능이 불러올 전자 산업의 새 영역에 발을 들이는 것"이라고 말했다. 소니는 1999년 아이보를 내놔 인기를 끌었지만 경영난으로 2006년 사업을 접었다. 당시 삼성전자에 